## 日内分时段的价量信息

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt

/home/aglv/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
cd /mnt/datadisk2/aglv/aglv/lab_aglv/forintern/

/mnt/datadisk2/aglv/aglv/lab_aglv/forintern


In [5]:
from DataDaily import DataDaily
dataDaily = DataDaily()
clo_day = dataDaily.adjclose
open_day = dataDaily.adjopen
night_ret = open_day / clo_day.shift(1) - 1

load ./data_daily.h5...


In [6]:
night_ret = night_ret.loc[20210101:20231231]
night_ret.index = [str(i) for i in night_ret.index]
night_ret

InstrumentID,BJ430017,BJ430047,BJ430090,BJ430139,BJ430198,BJ430300,BJ430418,BJ430425,BJ430476,BJ430478,...,SZ301589,SZ301591,SZ301596,SZ301600,SZ301603,SZ301606,SZ301607,SZ301608,SZ301611,SZ301618
20210104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20231225,0.000000,0.008027,0.004149,-0.001502,-0.037453,0.051922,-0.011405,-0.003475,0.003695,-0.007877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231226,0.000000,0.003249,0.019608,0.000000,0.036364,-0.042945,-0.009886,-0.033194,-0.016452,0.012734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231227,0.000000,0.008403,-0.033835,0.000709,-0.019556,-0.042351,-0.015267,-0.034416,-0.015493,-0.017228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231228,0.003309,-0.007910,0.020073,-0.014132,0.011184,-0.018659,-0.007064,-0.014213,0.000000,-0.007513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt

sys.path.append('/mnt/datadisk2/aglv/aglv/lab_aglv/forintern/')
from DataDaily import DataDaily

def factor_group(factor, group_number=10):
    '''按照因子值分组,grp1最小'''
    fg = factor.apply(
        lambda x: pd.qcut(x.rank(method='first'), q=group_number, labels=False) + 1, 
        axis=1)
        
    def _factor_group(factor_daily:pd.Series):
        dic = {f'group{str(g_value)}':factor_daily[factor_daily == g_value].index.tolist() for g_value in range(1, 1+group_number)}
        return pd.Series(dic)
    
    return {'group_stock':fg.apply(_factor_group, axis=1), 'group_index':fg}

def group_backtest(price:pd.DataFrame, factor:pd.DataFrame, group_number=10):
    '''
    分组回测,日度调仓,所有股票等权买入(买入各个股票的价值相同)  

    Paras:
    price:日频价格数据,index:Date,
    factor:日度因子值,index:Date,
    group_number:分组数,默认值为10, 组数大的因子值大

    Return:
    index:日期, col:组别, value: 指定日期下指定组数的收益率
    '''

    if 'Date' in factor.columns:
        factor = factor.set_index('Date')
    elif 'date' in factor.columns:
        factor = factor.set_index('date')

    factor = factor.shift(1).iloc[1:] # 第i日的因子作为第i+1日对应股票的调仓依据
    
    ret = price / price.shift(1)
    ret = ret.loc[factor.index, factor.columns].fillna(1)

    fg = factor_group(factor, group_number=group_number)['group_index']
    
    dic = {}
    for g_value in tqdm(range(1, 1+group_number)):
        is_in_group = fg.applymap(lambda x: 1 if x == g_value else 0)
        group_ret_daily = (ret * is_in_group).sum(axis=1) / is_in_group.sum(axis=1)
        dic[f'group{g_value}'] = group_ret_daily

    g_ret = pd.DataFrame(dic)
    # g_ret['g_mean'] = g_ret.mean(axis=1)
    g_ret['benchmark'] = ret.mean(axis=1)

    return g_ret

def position_backtest(price:pd.DataFrame, position:pd.DataFrame):
        '''
        有待完成
        用仓位数据来进行回测,
        price:日度价格数据
        position:对应的日度股票持仓数量
        '''
        holding = price * position
        holding_all = holding.sum(axis=1)
        ret = holding_all / holding_all - 1

        return ret

def group_backtest_plot(price:pd.DataFrame, factor:pd.DataFrame, group_number=5, save_path=None, is_zz1000=False, gap=1):
    '''
    分组回测
    Para:
    price:日频价格数据,index:Date,
    factor:日度因子值,index:Date,
    group_number:分组数,默认值为10, 组数大的因子值大
    '''

    if gap == 1:
        g_ret = group_backtest(price=price, factor=factor, group_number=group_number)
    else:
        g_ret = group_backtest_gapday(price=price, factor=factor, group_num=group_number, gap=gap) 

    g_ret.index = [str(i) for i in g_ret.index]
    g_ret_cumsum = g_ret.cumprod()

    if is_zz1000:
        index_data = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/index_data/close.csv')
        index_data.set_index('Unnamed: 0', inplace=True)
        zz1000 = index_data.loc[852, :]
        zz1000 = zz1000.loc[g_ret.index]
        zz1000_bt = zz1000 / zz1000.iloc[0]
        g_ret_cumsum['zz1000'] = zz1000_bt
    
    plt.style.use('seaborn')
    g_ret_cumsum.plot(figsize=(12, 6), linewidth=1)

    if is_zz1000:
        plt.plot(zz1000_bt, linewidth=2)

    plt.legend()

    if save_path is not None:
        plt.savefig(save_path, facecolor='white', dpi=300)
    plt.show()


def group_backtest_gapday(price:pd.DataFrame, factor:pd.DataFrame, group_num=5, gap=10):
    '''
    换仓期为gap日的回测系统
    '''

    def handle_factor(factor=factor, gap=gap):
        '''将因子值按gap日求平均得到新的间隔gap日的因子序列'''
        if 'Date' in factor.columns:
            factor.set_index('Date', inplace=True)
        elif 'date' in factor.columns:
            factor = factor.set_index('date')
        
        new_index = [factor.index[i] for i in range(0, len(factor.index), gap)]
        new_factor = pd.DataFrame(
            [factor.loc[new_index[i]:new_index[i+1]].mean() for i in range(len(new_index) - 1)], 
            index=new_index[:-1]
        )
        
        return new_factor, new_index
    
    new_factor, new_index = handle_factor(factor=factor)

    # 做axis=1的因子中性化 方便fillna(0)
    new_factor = ((new_factor.T - new_factor.T.mean()) / (new_factor.T.std() + 1e-10)).T.fillna(0)
    
    new_factor = new_factor.shift(1).iloc[1:] # shift意为 日期i作为日期i+1的分组指标
    
    ret = pd.DataFrame(
        [price.loc[new_index[i]] / (price.loc[new_index[i-1]] + 1e-10) for i in range(1, len(new_index))],
        index=new_index[1:]
    )
    ret = ret.loc[new_factor.index, new_factor.columns].fillna(1)

    fg = factor_group(new_factor, group_number=group_num)['group_index']
    
    dic = {}
    for g_value in tqdm(range(1, 1+group_num)):
        is_in_group = fg.applymap(lambda x: 1 if x == g_value else 0)
        group_ret_daily = (ret * is_in_group).sum(axis=1) / is_in_group.sum(axis=1)
        dic[f'group{g_value}'] = group_ret_daily

    g_ret = pd.DataFrame(dic)
    g_ret['benchmark'] = ret.mean(axis=1)

    return g_ret


def check_extrme_factor(factor:pd.DataFrame, date, rolling_day=10, is_large=True, check_num=10, check_period=5):
    '''
    检测某天的极端因子值的股票表现
    因子值按照向前rolling 10天的平均值进行排序 绘图显示前后5天的股票表现, 在jupyter中检测
    需要用到datadaily类获取日频价格数据
    '''

    if 'date' in factor.columns:
        factor = factor.set_index('date')
    elif 'Date' in factor.columns:
        factor = factor.set_index('Date')

    if date not in factor.index:
        raise ValueError('日期为非交易日')

    factor = factor.rolling(10).mean()
    factor_date = factor.loc[date]
    factor_date_sorted = factor_date.sort_values().dropna()

    if is_large:
        check_stock = factor_date_sorted.iloc[-check_num:].index
    else:
        check_stock = factor_date_sorted.iloc[:check_num].index
    
    close = datadaily.adjclose
    for stock in check_stock:
        date_id = close.index.get_loc(date)
        p_date = close.iloc[date_id-check_period:date_id+check_period+1, :].loc[:, stock]
        p_date.index = [str(i) for i in p_date.index]

        plt.plot(p_date)
        plt.title(stock)
        plt.show()


def factors_corr(factor_folder):
    '''检测因子之间相关性'''
    import glob, os
    import seaborn as sns

    csv_path = glob.glob(os.path.join(factor_folder, '*.csv'))
    new_path = []
    for p in csv_path:
        f_name = p.split('/')[-1][:-4]
        if 'relu' in f_name:
            new_path.append(p)
    
    new_path = new_path[:8]

    dfs = [pd.read_csv(f).set_index('date') for f in new_path]

    num = len(dfs)
    print(num)

    corrs = pd.DataFrame(index=range(num), columns=range(num))
    for i in range(num):
        print(i, new_path[i].split('/')[-1][:-4])
        for j in range(num):
            if j < i:
                corrs.iloc[i, j] = corrs.iloc[j ,i]
            elif j == i:
                corrs.iloc[i, j] = 1
            else:
                corrs.iloc[i, j] = dfs[i].corrwith(dfs[j]).mean()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(corrs.astype(float), annot=True, cmap='Greens', fmt=".2f")
    plt.title('Correlation with Means Heatmap')
    plt.show()

In [11]:
clo = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/close_21_23.h5')
clo

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20210104 09:31        9.64     14.01      8.82     12.35      2.86     76.46   
         09:32        9.60     13.97      8.84     12.36      2.84     76.17   
         09:33        9.60     14.00      8.83     12.37      2.85     76.10   
         09:34        9.58     13.90      8.83     12.33      2.85     75.74   
         09:35        9.58     13.88      8.94     12.28      2.85     75.36   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56        7.13     10.87      5.78     19.02      2.81     37.33   
         14:57        7.13     10.87      5.79     19.04      2.82     37.33   
         14:58        7.13     10.87      5.79     19.04      2.82     37.33   
         14:59        7.13     10.87      5.79     19.04      2.82     37.33   
         15:00        7.14     10.88      5.79     18.98      2.81     37.33   

                  SH600010  SH600011  SH600012  SH600015  ...  SZ301519  \
Date     EndTime                                          ...             
20210104 09:31        1.18      4.49      6.12      6.22  ...       NaN   
         09:32        1.17      4.47      6.16      6.20  ...       NaN   
         09:33        1.17      4.47      6.17      6.21  ...       NaN   
         09:34        1.17      4.46      6.16      6.21  ...       NaN   
         09:35        1.17      4.47      6.16      6.19  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20231012 14:56        1.73      7.81     10.83      5.72  ...     23.05   
         14:57        1.73      7.82     10.82      5.73  ...     23.06   
         14:58        1.73      7.82     10.81      5.73  ...     23.06   
         14:59        1.73      7.82     10.81      5.73  ...     23.06   
         15:00        1.72      7.81     10.83      5.71  ...     23.05   

                  SZ301520  SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  \
Date     EndTime                                                               
20210104 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56       65.80    101.99     84.05     44.24     41.43     61.38   
         14:57       65.65    101.81     84.08     44.28     41.39     61.37   
         14:58       65.64    101.81     84.10     44.28     41.39     61.34   
         14:59       65.64    101.81     84.10     44.28     41.39     61.34   
         15:00       65.64    101.90     84.29     44.35     41.40     61.33   

                  SZ301550  SZ301558  SZ301559  
Date     EndTime                                
20210104 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20231012 14:56       50.40     14.08     22.72  
         14:57       50.21     14.01     22.69  
         14:58       50.18     14.00     22.69  
         14:59       50.18     14.00     22.69  
         15:00       50.30     14.01     22.69  

[161118 rows x 5175 columns]

In [8]:
def inverse(p:pd.DataFrame, l_min=-30):
    l_ret = clo.groupby('Date', axis=0).apply(lambda x: x.iloc[l_min, :] / x.iloc[1, :] - 1) # 有些股票第0index伤没有数据
    r_ret = clo.groupby('Date', axis=0).apply(lambda x: x.iloc[-1, :] / x.iloc[l_min, :] -1)

    # sym = l_ret - r_ret
    sym = r_ret

    num = sym.shape[1] - sym.isna().sum(axis=1)
    inv_per = sym.sum(axis=1) / num
    
    return inv_per, sym, num

In [12]:
inv_per_30, sym_30, len_30 = inverse(clo, 30)

In [ ]:
ret_morn3 = sym_30.copy()

In [13]:
ret_afno30 = sym_30.copy()

In [ ]:
sym_30 = sym_30.replace(np.inf, np.nan)
sym_30

In [ ]:
def cVaR_series(series, c_level=0.9):
    right_VaR = series.quantile(c_level)
    # left_VaR = series.quantile(1- c_level)

    # X_low = pd.Series(series[series < left_VaR])
    X_high = pd.Series(series[series > right_VaR])

    # low_mean = X_low.mean() if len(X_low) > 0 else 0
    high_mean = X_high.mean() if len(X_high) > 0 else 0

    return high_mean

In [ ]:
cvar_ret_v1 = sym_30.rolling(30).progress_apply(cVaR_series)
cvar_ret_v1

In [ ]:
cvar_ret_v1.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/cvar_l-r_ret_r30.csv')

In [ ]:
sym_30_stack = sym_30.stack()
sym_30_stack

In [ ]:
da23 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/data2023.h5')
da23

In [ ]:
clo

In [ ]:
afno_jump_ret = clo.progress_apply(lambda x: x.loc[pd.IndexSlice[:, '13:10']] / x.loc[pd.IndexSlice[:, '11:20']] - 1)
afno_jump_ret

In [ ]:
night_ret = night_ret.loc[afno_jump_ret.index, afno_jump_ret.columns]
night_ret

In [ ]:
night_ret_stack

In [ ]:
afno_jump_ret

In [ ]:
afno_jump_ret.index

In [ ]:
night_ret.index

In [15]:
ret_afno30

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,0.010428,-0.007072,-0.009667,0.008130,0.000000,0.000000,0.008547,0.002227,-0.003289,0.003231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,0.012552,0.010241,0.019518,0.014540,0.003509,0.006904,0.000000,0.000000,0.001642,0.014587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,0.010288,-0.022383,-0.011655,0.020113,0.000000,-0.012866,-0.008547,0.018391,0.006339,0.003200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,0.005123,-0.038951,0.015663,-0.000805,-0.007092,-0.012884,-0.016807,0.004566,-0.012945,0.001605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.001016,0.011512,0.004902,0.005677,0.024823,-0.006797,0.008621,0.004494,-0.004902,-0.003190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,-0.006993,0.006289,0.000000,-0.006157,0.000000,0.009861,0.000000,-0.001269,0.007260,0.000000,...,-0.002877,-0.044331,-0.033232,-0.010736,0.008465,-0.020156,-0.019050,-0.012301,-0.040020,NaN
20231009,0.002853,-0.010999,0.001773,-0.006725,-0.007042,-0.002153,0.000000,-0.001269,0.013686,0.007092,...,-0.015365,0.003994,-0.005071,-0.017617,-0.016327,-0.005384,-0.015623,0.024482,-0.070288,NaN
20231010,-0.005674,-0.001843,0.007042,-0.000527,-0.003546,-0.007223,-0.005814,-0.006443,-0.010909,-0.001767,...,-0.021932,-0.037108,-0.000097,-0.010255,-0.029373,-0.019362,-0.021646,-0.028161,-0.063430,NaN


In [16]:
night_ret

InstrumentID,BJ430017,BJ430047,BJ430090,BJ430139,BJ430198,BJ430300,BJ430418,BJ430425,BJ430476,BJ430478,...,SZ301589,SZ301591,SZ301596,SZ301600,SZ301603,SZ301606,SZ301607,SZ301608,SZ301611,SZ301618
20210104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20231225,0.000000,0.008027,0.004149,-0.001502,-0.037453,0.051922,-0.011405,-0.003475,0.003695,-0.007877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231226,0.000000,0.003249,0.019608,0.000000,0.036364,-0.042945,-0.009886,-0.033194,-0.016452,0.012734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231227,0.000000,0.008403,-0.033835,0.000709,-0.019556,-0.042351,-0.015267,-0.034416,-0.015493,-0.017228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231228,0.003309,-0.007910,0.020073,-0.014132,0.011184,-0.018659,-0.007064,-0.014213,0.000000,-0.007513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
night_ret_stack = night_ret.rolling(30).mean().loc[ret_afno30.index, ret_afno30.columns].shift(1).T.stack()
ret_afno30.index.names = ['date']
ret_afno30_stack = ret_afno30.T.stack()
night_afno30_ret = pd.concat([night_ret_stack, ret_afno30_stack], axis=1)
night_afno30_ret

0         1
         date                  
SH600000 20210104 NaN  0.010428
         20210105 NaN  0.012552
         20210106 NaN  0.010288
         20210107 NaN  0.005123
         20210108 NaN -0.001016
...                ..       ...
SZ301558 20231010 NaN -0.063430
         20231011 NaN -0.014066
         20231012 NaN -0.004972
SZ301559 20231011 NaN -0.011964
         20231012 NaN -0.008737

[3133503 rows x 2 columns]

In [29]:
night_afno30_ret.index.names =['Stock', 'Date']

In [30]:
night_afno30_ret

0         1
Stock    Date                  
SH600000 20210104 NaN  0.010428
         20210105 NaN  0.012552
         20210106 NaN  0.010288
         20210107 NaN  0.005123
         20210108 NaN -0.001016
...                ..       ...
SZ301558 20231010 NaN -0.063430
         20231011 NaN -0.014066
         20231012 NaN -0.004972
SZ301559 20231011 NaN -0.011964
         20231012 NaN -0.008737

[3133503 rows x 2 columns]

In [31]:
night_afno30_ret_corr = (
    night_afno30_ret
    .groupby(level='Stock')
    .progress_apply(lambda x: x.iloc[:, 0].rolling(20).corr(x.iloc[:, 1]))
)
night_afno30_ret_corr

100%|██████████| 5175/5175 [00:20<00:00, 251.74it/s]


Stock     Stock     Date    
SH600000  SH600000  20210104   NaN
                    20210105   NaN
                    20210106   NaN
                    20210107   NaN
                    20210108   NaN
                                ..
SZ301558  SZ301558  20231010   NaN
                    20231011   NaN
                    20231012   NaN
SZ301559  SZ301559  20231011   NaN
                    20231012   NaN
Length: 3133503, dtype: float64

In [23]:
night_afno30_ret_corr.isna().sum()

138161

In [32]:
def drop_level1_index(data:pd.DataFrame):
    res = data.copy()
    res.index.names = ['a', 'InstrumentID', 'Date']
    id = res.index.droplevel('a')
    res.index = id
    del data

    return res.unstack(level='InstrumentID')

In [33]:
night_afno30_ret_corr_v2 = drop_level1_index(night_afno30_ret_corr)
night_afno30_ret_corr_v2

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,-0.265514,0.468564,-0.294323,0.197471,0.272593,0.193994,0.100986,-0.283510,0.132444,0.062027,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231009,-0.198604,0.447182,-0.241394,0.264452,0.123671,0.190941,0.131676,-0.383595,0.020865,0.254255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231010,-0.245310,0.215158,-0.104116,0.261927,0.043658,0.048408,0.102474,-0.401388,0.369411,0.227393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
night_afno30_ret_corr_v2.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/night_r30mean_s1_afno30min_ret_r20corr.csv')

In [ ]:
night_afno_jump_ret.columns = ['night', 'afno']
night_afno_jump_ret

In [ ]:
def draw_price_vol_unique(data:pd.DataFrame):
    '''
    data:MultiIndex对象, 一列价格一列成交量数据
    '''
    p_unique = data['Close'].unique()
    vol_unique = pd.Series(index=p_unique)
    for p in vol_unique.index:
        vol_unique.loc[p] = data['LastVolume'][data['Close'] == p].sum()

    plt.scatter(p_unique, vol_unique)
    

draw_price_vol(eg0103)

In [ ]:
eg0103 = da23.xs('20230111', level='Date')
eg0103 = eg0103.xs('SZ000023', level='InstrumentID')
eg0103 = eg0103.iloc[:, [1, 4]]
# eg0103['Close'] = (eg0103['Close'] - eg0103['Close'].mean()) / (eg0103['Close'].std())
# eg0103['LastVolume'] = (eg0103['LastVolume'] - eg0103['LastVolume'].mean()) / eg0103['LastVolume'].std()
# plt.scatter(eg0103.iloc[:, 0], eg0103.iloc[:, 1])
eg0103.iloc[:, 0].unique()

In [ ]:
def draw_price_vol_unique(data:pd.DataFrame):
    '''
    data:MultiIndex对象, 一列价格一列成交量数据
    '''
    p_unique = data['Close'].unique()
    vol_unique = pd.Series(index=p_unique)
    for p in vol_unique.index:
        vol_unique.loc[p] = data['LastVolume'][data['Close'] == p].sum()

    plt.scatter(p_unique, vol_unique)

In [ ]:
def draw_price_vol(data:pd.DataFrame):
    '''
    data:MultiIndex对象, 一列价格一列成交量数据
    '''
    # p_unique = data['Close'].unique()
    # vol_unique = pd.Series(index=p_unique)
    # for p in vol_unique.index:
    #     vol_unique.loc[p] = data['LastVolume'][data['Close'] == p].sum()

    plt.scatter(data.iloc[:, 0], data.iloc[:, 1])

In [ ]:
draw_price_vol(da23.xs('20230111', level='Date').xs('SZ000001', level='InstrumentID').iloc[:, [1, 4]])

In [ ]:
draw_price_vol_unique(da23.xs('20230111', level='Date').xs('SZ000001', level='InstrumentID').iloc[:, [1, 4]])

In [ ]:
# eg0103 = eg0103.iloc[:, [1, 4]]
eg0103['Close'] = (eg0103['Close'] - eg0103['Close'].mean()) / (eg0103['Close'].std())
eg0103['LastVolume'] = (eg0103['LastVolume'] - eg0103['LastVolume'].mean()) / (eg0103['LastVolume'].std())
eg0103

In [ ]:
plt.scatter(eg0103.iloc[:, 0], eg0103.iloc[:, 1])

In [130]:
cd /mnt/datadisk2/aglv/aglv/lab_aglv/forintern/

/mnt/datadisk2/aglv/aglv/lab_aglv/forintern


In [131]:
datadaily = DataDaily()
clo_day = datadaily.adjclose

load ./data_daily.h5...


In [ ]:
clo_day.index = [str(i) for i in clo_day.index]

In [ ]:
group_backtest_plot(clo_day, sym_30, gap=1)

In [ ]:
sym_30.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/min30_reverse_multi.csv')

In [ ]:
inv_per_30[inv_per_30 > 0.3]

In [ ]:
sym_30[inv_per_30 > 0.1]

In [ ]:
eg = clo.xs('20210107', level='Date')
eg2 = eg.iloc[-1, :] / eg.iloc[0, :] - 1
eg2.mean()

In [ ]:
index_close = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/data_index/close.csv')
hs300_clo = index_close.iloc[2, 1:]
hs300_clo.index = [str(i) for i in hs300_clo.index]
hs300_clo.index.names = ['date']
hs300_clo = hs300_clo.loc['20210101':]
hs300_clo

In [ ]:
index_op = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/data_index/open.csv')
hs300_op = index_op.iloc[2, 1:]
hs300_op.index = [str(i) for i in hs300_op.index]
hs300_op.index.names = ['date']
hs300_op = hs300_op.loc['20210101':]
hs300_op

In [ ]:
import scipy.stats as stats

In [ ]:
def asym_p_series(series):
    # series = np.array(series)
    # density_func = clt_density(series)
    # acc_func = clt_cdf(series)
    # density_series = density_func(series) # 收益率序列的密度函数 f(r)
    # acc_series = acc_func(series)
    # density_series = stats.norm.pdf(series, loc=np.mean(series), scale=np.std(series))
    # acc_series = stats.norm.cdf(series, loc=np.mean(series), scale=np.std(series))
    series = np.array(series)
    epsilon = 1e-10
    series += epsilon * np.random.randn(len(series))

    kde = stats.gaussian_kde(series)
    density_series = kde(series)
    acc_series = stats.rankdata(series) / len(series)
    # acc_series = np.array([integrate.romberg(kde, -10, i, tol=1e-2)[0] for i in series])

    return stats.spearmanr(density_series, acc_series)[0]

In [ ]:
import multiprocessing
shift_day = 0
r_win = 20
n_processing = 64

def _handle_stockly(data_stock: pd.DataFrame, r_win=r_win):
    # data_stock = (data_stock - data_stock.rolling(r_win).mean()) / (data_stock.rolling(r_win).std() + 1e-10)
    def _f(x: pd.Series):
            return x.std()

    return data_stock.rolling(r_win).apply(asym_p_series)

def process_group(group, r_win=r_win):
    instrument_id, group_data = group
    return (instrument_id, _handle_stockly(group_data, r_win))

def multiprocessing_generate_factor(data: pd.DataFrame, n_processes=n_processing):
    grouped = list(data.groupby(level='InstrumentID'))
    with multiprocessing.Pool(processes=n_processes) as pool:
        results = list(tqdm(pool.imap(process_group, grouped), total=len(grouped)))

    factor_list = []
    for instrument_id, factor in results:
        factor_df = pd.DataFrame(factor.values, index=factor.index.get_level_values('date'), columns=[factor.index[0][0]])
        factor_list.append(factor_df)

    factor = pd.concat(factor_list, axis=1)

    return factor

In [ ]:
ret_afno30_cvar_neg = sym_30.T.stack()
ret_afno30_cvar_neg.index.names = ['InstrumentID','date']
ret_afno30_cvar_neg = multiprocessing_generate_factor(ret_afno30_cvar_neg)
ret_afno30_cvar_neg


In [ ]:
group_backtest_plot(clo_day, ret_afno30_cvar_neg.iloc[20: :])

In [ ]:
ret_afno30_cvar_neg.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/afno30_ret_asym_neg_09_r20.csv')

In [ ]:
ret_morn3

In [ ]:
ret_afno30

In [ ]:
ret_morn30 = ret_morn3.T.stack()
ret_afno30 = ret_afno30.T.stack()
ret_difference = pd.concat([ret_morn30, ret_afno30], axis=1)
ret_difference

In [ ]:
ret_difference.index.names = ['InstrumentID', 'date']

In [ ]:
morn_afno_ret_difference = ret_difference.groupby(level='InstrumentID').progress_apply(lambda x: x.iloc[:, 0].rolling(20).corr(x.iloc[:, 1]))
morn_afno_ret_difference

In [ ]:
morn_afno_ret_difference.index.names = ['a', 'InstrumentID', 'date']
id = morn_afno_ret_difference.index.droplevel('a')
morn_afno_ret_difference.index = id
morn_afno_ret_difference

In [ ]:
morn_afno_ret_difference = morn_afno_ret_difference.T.unstack(level='InstrumentID')
morn_afno_ret_difference

In [ ]:
morn_afno_ret_difference.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/morn_afno_30_ret_r20corr.csv')

In [ ]:
morn_afno_ret_r20corr = pd.read_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/morn_afno_30_ret_r20corr.csv')
morn_afno_ret_r20corr

In [ ]:
# morn_afno_ret_r20corr.set_index('date', inplace=True)
morn_afno_ret_r20corr

In [ ]:
morn_afno_ret_r20corr.index = [str(i) for i in morn_afno_ret_r20corr.index]

In [ ]:
datadaily = DataDaily()

In [ ]:
vol = datadaily.volume
# vol.index = [str(i) for i in vol.index]
vol = vol.loc[morn_afno_ret_r20corr.index]
vol

In [ ]:
vol20 = vol / vol.rolling(20).mean()
vol20

In [ ]:
morn_afno_ret_r20corr

In [ ]:
morn_afno_ret_r20corr.index, vol20.index

In [ ]:
vol20std = vol20.rolling(20).std()
vol20std

In [ ]:
vol20std_morn_afno_ret_r20corr = vol20std * morn_afno_ret_r20corr
vol20std_morn_afno_ret_r20corr

In [ ]:
vol20std_morn_afno_ret_r20corr.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/vol20std_morn_afno_30_ret_r20corr.csv')

In [ ]:
clo

In [ ]:
eg = clo.xs('20210104', level='Date')
eg

In [ ]:
def joint_inday_ret(clo):
    ret_num = 10
    step = int(240 / ret_num)
    ret_arr = []
    for i in tqdm(range(ret_num)):
        res = clo.groupby(level='Date').apply(lambda x: x.iloc[step*(i+1), :] / x.iloc[step*i, :])
        ret_arr.append(res)
    
    return ret_arr

In [139]:
from concurrent.futures import ProcessPoolExecutor
ret_num = 10

def compute_return(clo, step, i):
    if i == ret_num - 1:
        return clo.groupby(level='Date').apply(lambda x: x.iloc[-1, :] / x.iloc[step*i, :])
    else:
        return clo.groupby(level='Date').apply(lambda x: x.iloc[step*(i+1), :] / x.iloc[step*i, :])

def joint_inday_ret(clo):
    step = int(240 / ret_num)
    ret_arr = []

    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(compute_return, clo, step, i) for i in range(ret_num)]
        for future in tqdm(futures):
            ret_arr.append(future.result().stack())

    return ret_arr

In [140]:
ret_arr = joint_inday_ret(clo)
ret_arr[0]

100%|██████████| 10/10 [03:52<00:00, 23.27s/it]


,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,0.993776,1.003569,1.060091,0.999190,1.003497,1.014125,0.983051,0.997773,0.998366,0.995177,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,0.989648,0.992769,0.943617,0.995165,0.996491,0.998566,0.991525,0.988789,0.985149,0.998384,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,1.005192,1.009538,0.986254,1.000000,1.000000,1.023754,0.991453,0.990847,1.012924,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,0.992901,0.985968,1.004831,0.984873,1.000000,0.990444,1.017241,0.997717,0.988800,0.993631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,1.002030,1.019577,0.978313,1.000812,1.003571,1.013302,0.991453,1.004545,1.003284,1.004808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,0.998601,0.988465,1.007286,1.001030,1.003534,0.997876,1.011696,0.997468,0.994599,0.998255,...,1.007450,0.929935,1.002956,1.003509,1.016168,1.009219,1.009381,0.994165,0.977723,NaN
20231009,0.990113,0.976703,1.001786,1.001034,1.000000,0.979937,1.000000,0.997459,0.989170,0.987698,...,0.997115,0.982766,0.992997,0.995251,0.986775,0.981162,0.985538,0.984211,0.929412,NaN
20231010,0.990182,1.009268,1.003515,0.990635,1.003546,1.011838,1.000000,0.978616,0.995483,1.000000,...,1.002109,0.960547,0.983313,1.016475,1.011646,1.000000,0.996380,1.145833,1.083571,NaN


In [148]:
stack_ret_arr = []
for ret in ret_arr:
    stack_ret_arr.append(ret.stack())

In [149]:
stack_ret_arr[0]

Date              
20210104  SH600000    0.993776
          SH600004    1.003569
          SH600006    1.060091
          SH600007    0.999190
          SH600008    1.003497
                        ...   
20231012  SZ301533    0.987654
          SZ301548    0.980667
          SZ301550    1.010161
          SZ301558    0.971724
          SZ301559    0.997391
Length: 3130989, dtype: float64

In [150]:
stack_ret = pd.concat(stack_ret_arr, axis=1)
stack_ret

0         1         2         3         4         5  \
Date                                                                            
20210104 SH600000  0.993776  1.000000  1.000000  1.001044  1.006257  1.002073   
         SH600004  1.003569  0.990043  1.002155  1.003584  1.000714  0.999286   
         SH600006  1.060091  0.994652  0.997849  1.004310  1.005365  0.991462   
         SH600007  0.999190  1.001621  1.002427  1.001614  1.003223  1.003213   
         SH600008  1.003497  0.996516  1.000000  1.000000  0.996503  1.003509   
...                     ...       ...       ...       ...       ...       ...   
20231012 SZ301533  0.987654  1.005637  0.996100  1.003915  0.999269  1.007561   
         SZ301548  0.980667  0.993264  1.002812  1.001319  1.002306  1.008381   
         SZ301550  1.010161  1.018146  0.976947  1.001388  1.003960  0.978107   
         SZ301558  0.971724  1.002839  0.992215  0.995007  1.009319  0.996449   
         SZ301559  0.997391  1.004795  0.989154  1.015351  0.991361  0.997386   

                          6         7         8         9  
Date                                                       
20210104 SH600000  1.001034  1.002066  1.000000  0.998969  
         SH600004  0.996429  1.000717  1.002149  1.003574  
         SH600006  0.990312  1.003261  1.013001  0.986096  
         SH600007  0.995997  0.999196  0.997586  1.000000  
         SH600008  1.000000  0.996503  1.003509  1.000000  
...                     ...       ...       ...       ...  
20231012 SZ301533  0.999758  1.001937  1.004592  0.995911  
         SZ301548  0.999022  1.006036  0.996270  0.998210  
         SZ301550  1.010284  0.999800  1.010781  0.993482  
         SZ301558  0.993585  0.999283  1.027997  0.978352  
         SZ301559  1.004369  0.993040  0.997810  0.996049  

[3130989 rows x 10 columns]

In [156]:
stack_ret

0         1         2         3         4         5  \
Date                                                                            
20210104 SH600000  0.993776  1.000000  1.000000  1.001044  1.006257  1.002073   
         SH600004  1.003569  0.990043  1.002155  1.003584  1.000714  0.999286   
         SH600006  1.060091  0.994652  0.997849  1.004310  1.005365  0.991462   
         SH600007  0.999190  1.001621  1.002427  1.001614  1.003223  1.003213   
         SH600008  1.003497  0.996516  1.000000  1.000000  0.996503  1.003509   
...                     ...       ...       ...       ...       ...       ...   
20231012 SZ301533  0.987654  1.005637  0.996100  1.003915  0.999269  1.007561   
         SZ301548  0.980667  0.993264  1.002812  1.001319  1.002306  1.008381   
         SZ301550  1.010161  1.018146  0.976947  1.001388  1.003960  0.978107   
         SZ301558  0.971724  1.002839  0.992215  0.995007  1.009319  0.996449   
         SZ301559  0.997391  1.004795  0.989154  1.015351  0.991361  0.997386   

                          6         7         8         9  
Date                                                       
20210104 SH600000  1.001034  1.002066  1.000000  0.998969  
         SH600004  0.996429  1.000717  1.002149  1.003574  
         SH600006  0.990312  1.003261  1.013001  0.986096  
         SH600007  0.995997  0.999196  0.997586  1.000000  
         SH600008  1.000000  0.996503  1.003509  1.000000  
...                     ...       ...       ...       ...  
20231012 SZ301533  0.999758  1.001937  1.004592  0.995911  
         SZ301548  0.999022  1.006036  0.996270  0.998210  
         SZ301550  1.010284  0.999800  1.010781  0.993482  
         SZ301558  0.993585  0.999283  1.027997  0.978352  
         SZ301559  1.004369  0.993040  0.997810  0.996049  

[3130989 rows x 10 columns]

In [166]:
from sklearn.decomposition import PCA
pca = PCA(2)
pca_ret = (
    stack_ret
    .groupby(level='Date')
    .progress_apply(lambda x: pd.DataFrame(pca.fit_transform(x), index=x.index))
)
pca_ret


100%|██████████| 671/671 [00:07<00:00, 87.47it/s] 


0         1
Date                                 
20210104 SH600000 -0.008473 -0.000376
         SH600004 -0.000533 -0.011993
         SH600006  0.055948 -0.016514
         SH600007 -0.002931 -0.000233
         SH600008  0.000527 -0.006450
...                     ...       ...
20231012 SZ301533 -0.010846 -0.002188
         SZ301548 -0.016947  0.002832
         SZ301550  0.012868  0.007678
         SZ301558 -0.024920  0.012088
         SZ301559 -0.001408 -0.005521

[3130989 rows x 2 columns]

In [169]:
fa_pca2 = pca_ret.iloc[:, 1].unstack()
fa_pca2

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.000376,-0.011993,-0.016514,-0.000233,-0.006450,-0.013659,0.010248,0.002004,0.005305,-0.004070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,-0.005495,-0.000779,-0.011989,-0.001137,-0.003087,-0.005413,-0.002568,-0.006746,0.003778,-0.003359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,0.007246,0.002904,-0.013227,0.008851,-0.003713,-0.013099,-0.000600,0.002474,0.036890,0.005734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,0.002506,0.013954,-0.006810,0.012043,0.001869,0.012711,0.004741,0.007637,0.012520,0.004465,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.010630,-0.000460,0.001130,0.001288,0.001469,-0.012753,0.005111,0.002658,-0.002008,-0.006870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,0.000707,-0.003945,0.002407,-0.002151,-0.002861,-0.005112,-0.002374,-0.001006,-0.001770,0.000867,...,0.004143,0.022100,-0.014780,-0.006113,0.006991,-0.009800,0.007292,0.035722,0.006237,NaN
20231009,-0.001038,-0.001713,0.002047,-0.001092,-0.003229,0.000370,-0.003687,0.003747,-0.000968,0.002630,...,-0.011390,-0.014984,-0.004180,0.001235,0.002212,-0.001118,-0.013810,0.012562,-0.018610,NaN
20231010,0.001972,-0.000312,-0.003582,-0.007107,-0.000452,-0.000996,-0.003191,0.002814,-0.000492,0.003315,...,0.001336,0.003329,-0.001868,-0.002133,-0.002108,-0.008961,-0.000891,-0.083701,-0.001981,NaN


In [170]:
fa_pca2.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/pca_10feat_to2_feat2.csv')

In [171]:
inday_ret = clo.groupby(level='Date').progress_apply(lambda x: x.iloc[-1, :] / x.iloc[0, :] - 1)
inday_ret

100%|██████████| 671/671 [00:52<00:00, 12.78it/s]  


,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,0.005187,0.002141,0.045351,0.004049,0.000000,0.013471,0.000000,0.002227,-0.009804,-0.001608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,0.002070,-0.001446,-0.055319,0.012087,0.003509,0.007954,-0.008475,-0.013453,0.006601,0.011309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,0.019730,-0.006603,-0.028637,0.018474,-0.003509,0.005841,-0.008547,0.013730,0.025848,0.006421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,-0.005071,-0.052437,0.018116,-0.011943,-0.007092,-0.020532,0.008621,0.004566,-0.024000,-0.006369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.002030,0.032106,-0.012048,0.007311,0.032143,0.000659,0.000000,0.015909,0.000000,0.001603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,-0.006993,-0.006211,0.012750,-0.002061,0.003534,0.006107,0.005848,-0.003797,-0.000900,-0.001745,...,0.004139,-0.093742,-0.033232,-0.008538,0.029014,-0.014059,-0.002724,0.006484,-0.061881,NaN
20231009,-0.007062,-0.033154,0.008929,-0.007752,-0.003534,-0.021383,0.000000,0.000000,0.002708,-0.001757,...,-0.022671,-0.003009,-0.010831,-0.020183,-0.023251,-0.023892,-0.031207,0.030737,-0.144118,NaN
20231010,-0.016830,0.003707,0.005272,-0.013007,-0.003546,-0.001614,-0.011561,-0.030189,-0.017164,-0.001767,...,-0.021932,-0.074961,-0.023758,0.005451,-0.019776,-0.023283,-0.025342,0.042500,0.033571,NaN


In [172]:
inday_ret.to_csv('/mnt/datadisk2/aglv/aglv/aglv_factor/Oct_factor/inday_ret.csv')